In [145]:
#pip install psycopg2-binary
#!pip install python-dateutil
#!pip install arrow


In [269]:
import pandas as pd
import json
import jsonlines
import re
import psycopg2
import sqlite3
from datetime import datetime
from psycopg2 import extras
from dateutil.parser import parse 
import arrow
import csv


In [18]:
# Fixing the JSON file
with open('fhir_encounters.jsonl', 'r') as file:
    data_str = ''.join(file.readlines())

index = data_str.find('sys_insert')

while index != -1:
    next_brace = data_str.find('}', index)
    if next_brace != -1:
        index = data_str.find('sys_insert', next_brace)
        if index != -1:
            data_str = data_str[:next_brace + 1] + ',' + data_str[next_brace + 1:]      
    else:
        break

json_fhir = "[{}]".format(data_str)
json_fhir = json.loads(json_fhir)

with open('fhir_encounters.json', 'w') as output_file:
    json.dump(json_fhir, output_file, indent=4)
    
json_fhir

[{'encounter': {'resourcetype': 'Bundle',
   'type': 'searchset',
   'total': 1,
   'entry': [{'fullurl': 'https://jade-forest-h.edu/sup-fhirproxy/api/FHIR/R4/Encounter/eDYzEfJtWqzyANXf5-1Efdx86jOMRWXizr-7iu.5QyG83',
     'resource': {'resourcetype': 'Encounter',
      'id': 'eDYzEfJtWqzyANXf5-1Efdx86jOMRWXizr-7iu.5QyG83',
      'status': 'arrived',
      'type': [{'coding': [{'system': 'urn:oid:1.2.998.1251.341.13.202.3.7.10.698084.30',
          'code': '2101020001',
          'display': 'Video Visit'}],
        'text': 'Video Visit'},
       {'coding': [{'system': 'urn:oid:1.2.998.1251.341.13.202.3.7.2.808267',
          'code': '200415',
          'display': 'Video Visit'}],
        'text': 'Video Visit'},
       {'coding': [{'system': 'urn:oid:1.2.998.1251.341.13.202.3.7.10.698084.18875',
          'code': '3',
          'display': 'Elective'}],
        'text': 'Elective'}],
      'subject': {'reference': 'Patient3015', 'display': 'Lily Young'},
      'participant': [{'period': {}

In [403]:
# Creating the relevant tables
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()


# System Inputs table - shows when the appointments were added to the DB
cursor.execute('''
    CREATE TABLE system_inputs (
        id SERIAL PRIMARY KEY,
        sys_insert TIMESTAMP WITH TIME ZONE
    )
''')


# Diagnoses table one-to-many
cursor.execute('''
    CREATE TABLE diagnosis (
        id SERIAL PRIMARY KEY,
        diagnosis_reference VARCHAR(50),
        diagnosis_name VARCHAR(100),
        system VARCHAR(50)
    )
''')


# Patients table
cursor.execute('''
    CREATE TABLE patients (
        id SERIAL PRIMARY KEY,
        patient_reference VARCHAR(50),
        patient_name VARCHAR(50)      
    )
''')

# Appointments table - Called "entries" initially but changed the table to a more descriptive name
cursor.execute('''
    CREATE TABLE appointments (
        id SERIAL PRIMARY KEY,
        status VARCHAR(50) CHECK (status IN ('arrived', 'finished')),
        appointment_url VARCHAR(300),
        start_time TIMESTAMP WITH TIME ZONE,
        end_time TIMESTAMP WITH TIME ZONE,
        system_inputs_id INTEGER,
        FOREIGN KEY (system_inputs_id) REFERENCES system_inputs(id),
        diagnosis_id INTEGER,
        FOREIGN KEY (diagnosis_id) REFERENCES diagnosis(id),
        patients_id INTEGER,
        FOREIGN KEY (patients_id) REFERENCES patients(id)

    )
''')

# Hospitals table
cursor.execute('''
    CREATE TABLE hospitals (
        id SERIAL PRIMARY KEY,
        hospital_reference VARCHAR(50),
        hospital_name VARCHAR(50)
    )
''')

# A patient might change hospital/location mid-appointment but as I wasn't sure so I relied on the fact I got the hospital in an array labeled location. So I left this also many-to-many to take care of that option.
# Many to many hospitals-appointments bridge table
cursor.execute('''
CREATE TABLE appointments_hospitals (
    appointment_id INTEGER,
    hospital_id INTEGER,
    PRIMARY KEY (appointment_id, hospital_id),
    FOREIGN KEY (appointment_id) REFERENCES appointments(id),
    FOREIGN KEY (hospital_id) REFERENCES hospitals(id)
    )
''')




# Doctors table
cursor.execute('''
    CREATE TABLE doctors (
        id SERIAL PRIMARY KEY,
        dr_reference VARCHAR(50),
        dr_name VARCHAR(50)
    )
''')

# Like in hospitals I got this data in an array and its possible one appointment has a few doctors to advise the patient.
# Many to many doctors-appointments bridge table
cursor.execute('''
CREATE TABLE doctors_appointments (
    appointment_id INTEGER,
    doctors_id INTEGER,
    PRIMARY KEY (appointment_id, doctors_id),
    FOREIGN KEY (appointment_id) REFERENCES appointments(id),
    FOREIGN KEY (doctors_id) REFERENCES doctors(id)
    )
''')





# Appointment Location table (To save the code/system/display of the appointment in case its needed)
cursor.execute('''
    CREATE TABLE locations (
        id SERIAL PRIMARY KEY,
        loc_system VARCHAR(100),
        loc_code VARCHAR(50),
        loc_display VARCHAR(50),
        appointment_id INTEGER,
        FOREIGN KEY (appointment_id) REFERENCES appointments(id)

    )
''')

conn.commit()
conn.close()



In [404]:
# Extracting the data from the json file
items_to_insert = []

for entry in json_fhir:
    itemToInsert = {}
    sys_insert = entry['sys_insert']
    itemToInsert['system_inputs']=sys_insert 
    entries = entry['encounter']['entry']
    visitsArr = []
    for visit in entries:
        visits = {}
        appointment_url = visit['fullurl']
        
        fixedVisit = visit['resource']
        appointment = {}
        if 'status' in fixedVisit:
            appointment_status = fixedVisit['status']
            appointment['appointment_url']=appointment_url
            appointment['appointment_status']= appointment_status
        else:
            appointment['appointment_url']=appointment_url
            appointment['appointment_status']= None
              

        if 'start' in fixedVisit['period'] and '_end' in fixedVisit['period']:
            start_time = fixedVisit['period']['start']
            end_time = fixedVisit['period']['_end']
        elif 'start' in fixedVisit['period']:
            start_time = fixedVisit['period']['start']
            end_time = None
        elif '_end' in fixedVisit['period']:
            start_time = None
            end_time = fixedVisit['period']['_end']
        else:
            start_time = end_time = None

        appointment= {**appointment, 'start_time': start_time, 'end_time': end_time}
        visits['appointment'] = appointment
        
        hospitals = fixedVisit['location']
        hospitalArr = []
        for hospital in hospitals:
            hosReference = hospital['location']['reference']
            hosDisplay = hospital['location']['display']
            hospitalArr.append({"hospital_reference": hosReference,"hospital_name":hosDisplay})
        visits['hospitals']=hospitalArr
        
        participants = fixedVisit['participant']
        drsArr = []
        for participant in participants:
            participant_reference = participant.get('reference', False)
            if participant_reference:
                isDrMatch = re.findall(r'\b(?:Dr|dr)\W*\d*\b', participant_reference)
                if isDrMatch:
                    parDisplay = participant['display']
                    drsArr.append({"dr_reference": participant_reference,"dr_name":parDisplay})
        visits['doctors']=drsArr
        
        locations = fixedVisit.get('type', [])
        locationArr = []
        for location in locations:
            coding = location.get('coding', [])
            for obj in coding:
                locSys = obj.get('system')
                locCode = obj.get('code')
                locDisplay = obj.get('display')

                if locSys and locCode and locDisplay:
                    locationArr.append({"loc_system": locSys,"loc_code": locCode,"loc_display": locDisplay})
                else:
                    locationArr.append({"loc_system": None,"loc_code": None,"loc_display": None})
        visits['locations'] = locationArr

        
        visits['diagnoses'] = {'diagnosis_reference': fixedVisit['diagnosis']['reference'], 'diagnosis_name':fixedVisit['diagnosis']['display'], 'system':fixedVisit['diagnosis']['system']}
        visits['patients'] = {'patient_reference': fixedVisit['subject']['reference'], 'patient_name':fixedVisit['subject']['display']}
        visitsArr.append(visits)
    itemToInsert['visitsArr']=visitsArr
    items_to_insert.append(itemToInsert)

items_to_insert

[{'system_inputs': '2023-11-26 11:35:38.404169 UTC',
  'visitsArr': [{'appointment': {'appointment_url': 'https://jade-forest-h.edu/sup-fhirproxy/api/FHIR/R4/Encounter/eDYzEfJtWqzyANXf5-1Efdx86jOMRWXizr-7iu.5QyG83',
     'appointment_status': 'arrived',
     'start_time': '2023-11-22T03:00:00Z',
     'end_time': '2023-11-22T03:15:00Z'},
    'hospitals': [{'hospital_reference': 'Location/emFR3fX6yqT.KL--hFHkxMw3',
      'hospital_name': 'Jade Forest Hospital Center'}],
    'doctors': [{'dr_reference': 'Practitioner/Dr2002',
      'dr_name': 'Dr. Benjamin Carter'}],
    'locations': [{'loc_system': 'urn:oid:1.2.998.1251.341.13.202.3.7.10.698084.30',
      'loc_code': '2101020001',
      'loc_display': 'Video Visit'},
     {'loc_system': 'urn:oid:1.2.998.1251.341.13.202.3.7.2.808267',
      'loc_code': '200415',
      'loc_display': 'Video Visit'},
     {'loc_system': 'urn:oid:1.2.998.1251.341.13.202.3.7.10.698084.18875',
      'loc_code': '3',
      'loc_display': 'Elective'}],
    'diag

In [405]:
# Adding the data to the tables that were created

systemDataToInsert = []

for index,item in enumerate(items_to_insert):
    timestamp_str = item['system_inputs'] 
    parts = timestamp_str.split('.')
    date_time = parts[0]  
    
    milliseconds = parts[1].split()[0] if len(parts) > 1 else '000000' 
    if len(milliseconds) < 6:
        milliseconds = milliseconds + "0"

    timestamp_str = f"{date_time}.{milliseconds} {'UTC' if len(parts) > 1 else ''}"
    timestamp = arrow.get(timestamp_str, 'YYYY-MM-DD HH:mm:ss.SSSSSS ZZZ').datetime

    systemDataToInsert.append({'sys_insert': timestamp})

conn = psycopg2.connect(
dbname="FHIR",
user="postgres",
password="password",
host="localhost",
port="5432")

cursor = conn.cursor()

values = [(item['sys_insert'],) for item in systemDataToInsert]

sql = 'INSERT INTO public.system_inputs (sys_insert) VALUES %s RETURNING id'

results = extras.execute_values(cursor, sql, values, fetch=True)

conn.commit()
conn.close()


In [406]:

ind_list = [result[0] for result in results]

appointmentsToInsert = []

for index, item in enumerate(items_to_insert):
    visitArr = item['visitsArr']
    for visit in visitsArr:
        appointments = visit['appointment']
        appointmentsToInsert.append({'system_inputs_id': ind_list[index],'appointment_url': appointments['appointment_url'],
                                     'status':appointments['appointment_status'], 'start_time':appointments['start_time'],
                                    'end_time':appointments['end_time']})    

 
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO appointments (status, appointment_url, start_time, end_time, system_inputs_id) VALUES %s RETURNING id'
values = [(
        appointment['status'],
        appointment['appointment_url'],
        appointment['start_time'],
        appointment['end_time'],
        appointment['system_inputs_id']
    )
    for appointment in appointmentsToInsert]

idOfAppointments = extras.execute_values(cursor, sql, values, fetch=True)
conn.commit()
conn.close()


In [407]:
ind_list = [ind[0] for ind in idOfAppointments]


doctorValue = []
# [{},{}]
appointmentIdToConnToDr = []
# [[5,3],[1,2,4]]

#  index count helps us see how many appointments we iterated through in order to get the correct index of the appointment id inside the above array
indexCount=0 
for index,item in enumerate(items_to_insert):
    visitArr = item['visitsArr']
    for visitIndex, visit in enumerate(visitsArr):
        doctors= visit['doctors']
        for dr in doctors:
            index = next((idx for idx, drToCheck in enumerate(doctorValue) if drToCheck['dr_reference'] == dr['dr_reference']), None)
            if index is not None:
                if len(appointmentIdToConnToDr) > index:
                    appointmentIdToConnToDr[index].append(ind_list[indexCount])
                else:
                    print('error not enough indexes - Should not happen')
            else:
                doctorValue.append({'dr_reference': dr['dr_reference'], 'dr_name':dr['dr_name']})
                appointmentIdToConnToDr.append([ind_list[indexCount]])
        indexCount+=1
       
 
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO doctors (dr_reference, dr_name) VALUES %s RETURNING id'
values = [(
        doctor['dr_reference'],
        doctor['dr_name']
    )
    for doctor in doctorValue]

idOfDoctors = extras.execute_values(cursor, sql, values, fetch=True)
conn.commit()




idOfDoctors

[(1,), (2,), (3,), (4,), (5,)]

In [408]:
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

# preparing the data to insert into the bridge
dr_ind_list = [ind[0] for ind in idOfDoctors]

bridgeValuesToEnter= []

for indexOfDrId, appntArr in enumerate(appointmentIdToConnToDr):
    for appntId in appntArr:
        bridgeValuesToEnter.append({'dr_id':dr_ind_list[indexOfDrId],'appnt_id':appntId})
        
        
sqlBridge = "INSERT INTO doctors_appointments (appointment_id, doctors_id) VALUES %s"

valuesBridge = [(
        bridgeVal['appnt_id'],
        bridgeVal['dr_id']
    )
    for bridgeVal in bridgeValuesToEnter]

extras.execute_values(cursor, sqlBridge, valuesBridge)
conn.commit()

In [409]:
ind_list = [ind[0] for ind in idOfAppointments]

hospitalValue = []
# [{},{}]
appointmentIdToConnToHos = []
# [[5,3],[1,2,4]]

indexCount=0
for item in items_to_insert:
    visitArr = item['visitsArr']
    for visitIndex, visit in enumerate(visitsArr):
        hospitals= visit['hospitals']
        for hospital in hospitals:
            index = next((idx for idx, hosToCheck in enumerate(hospitalValue) if hosToCheck['hospital_reference'] == hospital['hospital_reference']), None)
            if index is not None:
                if len(appointmentIdToConnToHos) > index:
                    appointmentIdToConnToHos[index].append(ind_list[indexCount])
                else:
                    print('error not enough indexes - Should not happen')
            else:
                hospitalValue.append({'hospital_reference': hospital['hospital_reference'], 'hospital_name':hospital['hospital_name']})
                appointmentIdToConnToHos.append([ind_list[indexCount]])
        indexCount+=1
       

 
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO hospitals (hospital_reference, hospital_name) VALUES %s RETURNING id'
values = [(
        hospital['hospital_reference'],
        hospital['hospital_name']
    )
    for hospital in hospitalValue]

idOfHospitals = extras.execute_values(cursor, sql, values, fetch=True)
conn.commit()


In [410]:
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

# preparing the data to insert into the bridge
hos_ind_list = [ind[0] for ind in idOfHospitals]

bridgeValuesToEnter= []

for indexOfHosId, appntArr in enumerate(appointmentIdToConnToHos):
    for appntId in appntArr:
        bridgeValuesToEnter.append({'hos_id':hos_ind_list[indexOfHosId],'appnt_id':appntId})
        
        
sqlBridge = "INSERT INTO appointments_hospitals (appointment_id, hospital_id) VALUES %s"

valuesBridge = [(
        bridgeVal['appnt_id'],
        bridgeVal['hos_id']
    )
    for bridgeVal in bridgeValuesToEnter]

extras.execute_values(cursor, sqlBridge, valuesBridge)
conn.commit()

In [411]:
ind_list = [ind[0] for ind in idOfAppointments]

locationValue = []

indexCount=0
for index,item in enumerate(items_to_insert):
    visitArr = item['visitsArr']
    for visitIndex, visit in enumerate(visitsArr):
        locations= visit['locations']
        for loc in locations:
            locationValue.append({'loc_system': loc['loc_system'], 'loc_code':loc['loc_code'],'loc_display':loc['loc_display'], 'appointment_id':ind_list[indexCount]})
        indexCount+=1
       

conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO locations (loc_system, loc_code, loc_display, appointment_id) VALUES %s RETURNING id'
values = [(
        loc['loc_system'],
        loc['loc_code'],
        loc['loc_display'],
        loc['appointment_id']
    )
    for loc in locationValue]

extras.execute_values(cursor, sql, values, fetch=True)
conn.commit()



In [412]:
ind_list = [ind[0] for ind in idOfAppointments]

diagnosesUniqueValue = []
# [{},{}]
repeatDiagnosesAccordingToAppnt = []
# mantains index of the id correlated to with id of appointment

indexCount = 0
for item in items_to_insert:
    visitArr = item['visitsArr']
    for visit in visitsArr:
        diagnosis= visit['diagnoses']
        diagObj = {'diagnosis_reference': diagnosis['diagnosis_reference'], 'diagnosis_name':diagnosis['diagnosis_name'],'system':diagnosis['system']}
        index = next((idx for idx, diagToCheck in enumerate(diagnosesUniqueValue) if diagToCheck['diagnosis_reference'] == diagnosis['diagnosis_reference']), None)
        if index is None:
            diagnosesUniqueValue.append(diagObj)
        repeatDiagnosesAccordingToAppnt.append(diagObj)
    indexCount+=1
        
        
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO diagnosis (diagnosis_reference, diagnosis_name, system) VALUES %s RETURNING id'
values = [(
        diag['diagnosis_reference'],
        diag['diagnosis_name'],
        diag['system']
    )
    for diag in diagnosesUniqueValue]

idOfDiag= extras.execute_values(cursor, sql, values, fetch=True)


conn.commit()



In [413]:
appnt_id_list = [ind[0] for ind in idOfAppointments]
diag_id_list = [ind[0] for ind in idOfDiag]

# print(diagnosesValue) the order it was entered into the db
print(len(appnt_id_list))
print(len(diag_id_list))
print(len(diagnosesUniqueValue))
arrayOfAppntConToDiag =  [[] for _ in range(len(diagnosesUniqueValue))] # empty array according to length to order according to diag id with its index
# indexed based on diagnoses with array of appnt connected to
for indexOfAppnt, diag in enumerate(repeatDiagnosesAccordingToAppnt):
    indexOfEnteredToDbDiagnosis = next((index for index, diagnos in enumerate(diagnosesUniqueValue) if diagnos['diagnosis_reference'] == diag['diagnosis_reference']), None)
    if indexOfEnteredToDbDiagnosis is not None:
        arrayOfAppntConToDiag[indexOfEnteredToDbDiagnosis].append(appnt_id_list[indexOfAppnt])
    else:
        print('error should not happen, didnt find index of referenced diagnosis')
        

updAppnt = []

for indexOfDiag, arrAppnt in enumerate(arrayOfAppntConToDiag):
    for appntIds in arrAppnt:
        updAppnt.append({'diagId':diag_id_list[indexOfDiag],'appntId':appntIds})

conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

values = [(apnt['diagId'], apnt['appntId']) for apnt in updAppnt]

sql = 'UPDATE appointments SET diagnosis_id = %s WHERE id = %s'

extras.execute_batch(cursor, sql, values)


conn.commit()


1000
11
11


In [414]:
ind_list = [ind[0] for ind in idOfAppointments]

patientsUniqueValue = []
# [{},{}]
repeatPatientsAccordingToAppnt = []
# mantains index of the id correlated to with id of appointment

indexCount = 0
for item in items_to_insert:
    visitArr = item['visitsArr']
    for visit in visitsArr:
        patient= visit['patients']
        patientObj = {'patient_reference': patient['patient_reference'], 'patient_name':patient['patient_name']}
        index = next((idx for idx, patToCheck in enumerate(patientsUniqueValue) if patToCheck['patient_reference'] == patientObj['patient_reference']), None)
        if index is None:
            patientsUniqueValue.append(patientObj)
        repeatPatientsAccordingToAppnt.append(patientObj)
    indexCount+=1
        
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

sql = 'INSERT INTO patients (patient_reference, patient_name) VALUES %s RETURNING id'
values = [(
        pt['patient_reference'],
        pt['patient_name']
    )
    for pt in patientsUniqueValue]

patientIds = extras.execute_values(cursor, sql, values, fetch=True)


conn.commit()

In [415]:
appnt_id_list = [ind[0] for ind in idOfAppointments]
pat_id_list = [ind[0] for ind in patientIds]



arrayOfAppntConToPat =  [[] for _ in range(len(patientsUniqueValue))] # empty array according to length to order according to patient id with its index
# indexed based on patients with array of appnt connected to
for indexOfAppnt, patient in enumerate(repeatPatientsAccordingToAppnt):
    indexOfEnteredToDbPatients = next((index for index, patToCheck in enumerate(patientsUniqueValue) if patToCheck['patient_reference'] == patient['patient_reference']), None)
    if indexOfEnteredToDbPatients is not None:
        arrayOfAppntConToPat[indexOfEnteredToDbPatients].append(appnt_id_list[indexOfAppnt])
    else:
        print('error should not happen, didnt find index of referenced diagnosis')
        

updAppnt = []

for indexOfPat, arrAppnt in enumerate(arrayOfAppntConToPat):
    for appntIds in arrAppnt:
        updAppnt.append({'patId':pat_id_list[indexOfPat],'appntId':appntIds})
        
print(updAppnt)
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

values = [(apnt['patId'], apnt['appntId']) for apnt in updAppnt]

sql = 'UPDATE appointments SET patients_id = %s WHERE id = %s'


extras.execute_batch(cursor, sql, values)


conn.commit()


[{'patId': 1, 'appntId': 1}, {'patId': 1, 'appntId': 21}, {'patId': 1, 'appntId': 41}, {'patId': 1, 'appntId': 61}, {'patId': 1, 'appntId': 81}, {'patId': 1, 'appntId': 101}, {'patId': 1, 'appntId': 121}, {'patId': 1, 'appntId': 141}, {'patId': 1, 'appntId': 161}, {'patId': 1, 'appntId': 181}, {'patId': 1, 'appntId': 201}, {'patId': 1, 'appntId': 221}, {'patId': 1, 'appntId': 241}, {'patId': 1, 'appntId': 261}, {'patId': 1, 'appntId': 281}, {'patId': 1, 'appntId': 301}, {'patId': 1, 'appntId': 321}, {'patId': 1, 'appntId': 341}, {'patId': 1, 'appntId': 361}, {'patId': 1, 'appntId': 381}, {'patId': 1, 'appntId': 401}, {'patId': 1, 'appntId': 421}, {'patId': 1, 'appntId': 441}, {'patId': 1, 'appntId': 461}, {'patId': 1, 'appntId': 481}, {'patId': 1, 'appntId': 501}, {'patId': 1, 'appntId': 521}, {'patId': 1, 'appntId': 541}, {'patId': 1, 'appntId': 561}, {'patId': 1, 'appntId': 581}, {'patId': 1, 'appntId': 601}, {'patId': 1, 'appntId': 621}, {'patId': 1, 'appntId': 641}, {'patId': 1, 'a

In [416]:
#First query

conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

cursor.execute("SELECT DISTINCT patient_reference, patient_name FROM patients")

patient_info = cursor.fetchall()

cursor.close()
conn.close()

with open('patient_info.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Patient Reference', 'Patient Name'])
    for patient in patient_info:
        writer.writerow(patient)

print("CSV file 'patient_info.csv' has been created with patient information.")


CSV file 'patient_info.csv' has been created with patient information.


In [424]:
#Second query

conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

cursor.execute("""
    SELECT patients.patient_name, COUNT(appointments.id) AS appointment_count
    FROM patients
    JOIN appointments ON patients.id = appointments.patients_id
    GROUP BY patients.patient_name
""")

patient_info = cursor.fetchall()

cursor.close()
conn.close()

with open('appointments_count.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Patient Name', 'Appointment Count'])
    for patient in patient_info:
        writer.writerow(patient)

print("CSV file 'appointments_count.csv' has been created with patient appointment counts.")


CSV file 'appointments_count.csv' has been created with patient appointment counts.


In [426]:
#Third query

conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

cursor.execute('''
    SELECT d.dr_name AS practitioner_name, COUNT(da.appointment_id) AS encounter_count
    FROM doctors d
    JOIN doctors_appointments da ON d.id = da.doctors_id
    GROUP BY d.dr_name
    ORDER BY encounter_count DESC
    LIMIT 1
''')

result = cursor.fetchone()

if result:
    practitioner_name, encounter_count = result
    print(f"The practitioner '{practitioner_name}' has the highest number of encounters: {encounter_count}")
else:
    print("No data found.")

cursor.close()
conn.close()


The practitioner 'Dr. Sophia Garcia-Richardsנ' has the highest number of encounters: 400


In [428]:
conn = psycopg2.connect(
    dbname="FHIR",
    user="postgres",
    password="password",
    host="localhost",
    port="5432")

cursor = conn.cursor()

cursor.execute('''
    SELECT month, condition, condition_count
    FROM (
        SELECT EXTRACT(MONTH FROM start_time) AS month,
               diagnosis.diagnosis_name AS condition,
               COUNT(*) AS condition_count,
               ROW_NUMBER() OVER (PARTITION BY EXTRACT(MONTH FROM start_time) ORDER BY COUNT(*) DESC) AS rn
        FROM appointments
        JOIN diagnosis ON appointments.diagnosis_id = diagnosis.id
        GROUP BY month, condition
    ) sub
    WHERE rn = 1
    ORDER BY month
''')

results = cursor.fetchall()

csv_filename = "common_conditions_per_month.csv"

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Month', 'Condition', 'Condition Count'])

    for row in results:
        month, condition, condition_count = row
        csv_writer.writerow([month, condition, condition_count])

print(f"Results saved to '{csv_filename}'")

cursor.close()
conn.close()


Results saved to 'common_conditions_per_month.csv'
